In [1]:
from supriya import *
server = Server()
server.boot()

<Server: udp://127.0.0.1:57751, 8i8o>

In [2]:
with SynthDefBuilder(amplitude = 0.5, 
                frequency = 220.0, 
                      gate = 1.0,
                      b1 = 0,
                      b2 = 1) as builder:     
    source = ugens.SinOsc.ar(frequency=builder['frequency'],)
    envelope = ugens.EnvGen.kr(done_action=DoneAction.FREE_SYNTH, 
                               envelope=Envelope.asr(), 
                               gate=builder['gate'],)
    source = source * builder['amplitude'] * envelope
    out = ugens.Out.ar(bus=(builder['b1'], builder['b2']), source=source,)
    synthdef = builder.build().allocate() # this is how one creates a synthdef. Equivalent of .add, I guess
server.sync()

<Server: udp://127.0.0.1:57751, 8i8o>

In [5]:
synth = Synth(synthdef)

In [8]:
group = Group()
group = group.allocate()

In [7]:
group.append(synth)

In [8]:
synth['gate'] = 0

## We want to build a synthdef that will output to whatever a buffer of a fixed length, then we want to write to that buffer, and then play back that buffer

In [3]:
from supriya.ugens import *

In [116]:
b = Buffer()
b = b.allocate(frame_count=44100, channel_count=2)

In [168]:
def newSynthDefBufWr(buffer):
    with SynthDefBuilder() as builder:
        phase = Phasor.ar(rate = BufRateScale.kr(buffer_id=buffer.buffer_id),
                          start = 0,
                          stop = BufFrames.kr(buffer_id=buffer.buffer_id))
        source = SoundIn.ar(bus=(0,1))
#         source = (SinOsc.ar(220), SinOsc.ar(220*(3/4)))
        source = BufWr.ar(
            buffer_id=b.buffer_id,
            loop=1,
            phase=phase,
            source=source
        )
        out = Out.ar(bus=4, source=source)
    synthdef = builder.build().allocate()
    server.sync()
    return synthdef

In [169]:
bufwr = newSynthDefBufWr(b)

In [170]:
def newSynthDefBufRd(buffer):
    with SynthDefBuilder() as builder:
        phase = Phasor.ar(rate = BufRateScale.kr(buffer_id=buffer.buffer_id),
                          start = 0,
                          stop = BufFrames.kr(buffer_id=buffer.buffer_id))
        source = BufRd.ar(
            buffer_id=buffer.buffer_id,
            channel_count=buffer.channel_count,
            interpolation=2,
            loop=1,
            phase=phase
        )
        out = Out.ar(bus=(0,1), source=source)
    synthdef = builder.build().allocate()
    server.sync()
    return synthdef

In [171]:
bufrd = newSynthDefBufRd(b)

In [172]:
group = Group()
group = group.allocate()

In [173]:
wrsynth = Synth(bufwr)
rdsynth = Synth(bufrd)

In [174]:
group.append(wrsynth)

In [175]:
group.append(rdsynth)

In [176]:
group.free()

<Group: ???>

In [148]:
rdsynth.free()

In [149]:
wrsynth.free()